In [9]:
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import random
import numpy as np
import csv
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
t_train = pd.read_csv("titanic_1030-2.csv",sep=",")
t_train.head()

,pclass,survived,sex,age,if child,fare,is port,is A,boat,sibsp,parch,if is port and man
0,1,0,1,39.0,0,0,0.0,0,0.0,0,0,0
1,1,0,1,0.0,0,0,0.0,0,0.0,0,0,0
2,1,0,1,0.0,0,0,0.0,0,0.0,0,0,0
3,1,0,1,40.0,0,0,0.0,0,0.0,0,0,0
4,1,0,1,0.0,0,0,0.0,0,0.0,0,0,0


In [5]:
train = t_train.sample(frac=0.8)
test = t_train.loc[~t_train.index.isin(train.index)]

In [6]:
#drop=["survived"]
drop=["survived","sibsp","parch","is A","is port"]
x_train = train.copy().drop(drop,axis=1).astype(np.float32).as_matrix().reshape(-1,7)
y_train = train["survived"].copy().astype(np.float32).as_matrix().reshape(-1,1)

x_test = test.copy().drop(drop,axis=1).astype(np.float32).as_matrix().reshape(-1,7)
y_test = test["survived"].copy().astype(np.float32).as_matrix().reshape(-1,1)

In [20]:
class LR(nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        self.hidden1 = nn.Linear(7, 10)
        self.dropout = nn.Dropout(0.3)
        self.relu1 = nn.ReLU()
        self.hidden2 = nn.Linear(10, 15)
        self.dropout = nn.Dropout(0.3)
        self.relu2 = nn.ReLU()
        self.hidden3 = nn.Linear(15,1)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x2 = F.relu(self.hidden2(x))
        out = self.hidden3(x2)
        return out
model = LR()
loss_func = nn.MSELoss()
learning_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
iters=0

In [21]:
epochs = 10000
for epoch in range(epochs):
    epoch += 1

    # convert to variables
    x = Variable(torch.from_numpy(x_train), requires_grad = True)
    y = Variable(torch.from_numpy(y_train) , requires_grad = False)

    # clear gradient w.r.t. parameters
    optimizer.zero_grad()

    # forward to get output
    prediction = model(x)

    # calculate loss
    loss = loss_func(prediction, y)

    # backward to get gradient
    loss.backward()

    # update parameters
    optimizer.step()
    iters+=1
    
    if epoch % 5 == 0:       
#        print("epoch %d, loss %.8f" % (epoch, loss.data[0]))
        if iters % 1000 == 0:
            correct = 0
            total = 0
            for i in range(len(x_test)):
                xt = Variable(torch.from_numpy(x_test[i]), requires_grad = False)
				# predict with current weights
                yt = Variable(torch.from_numpy(y_test[i]), requires_grad = False)
                outputs = model(xt)
                output = outputs.data.numpy()
                if output[0] > 0.5:
                    predicted = torch.FloatTensor([1])
                else:
                    predicted = torch.FloatTensor([0])
				#
                total += 1
                correct += (predicted == yt.data)
            correct = int(correct.numpy()[0])
            accuracy = 100 * correct / total
            print("Iteration: {}. Loss: {}. Accuracy: {}".format(iters, loss, accuracy))


#torch.save(model.state_dict(), "practice_titanic_model1.pkl")

Iteration: 1000. Loss: Variable containing:
 0.2196
[torch.FloatTensor of size 1]
. Accuracy: 64.5
Iteration: 2000. Loss: Variable containing:
 0.1897
[torch.FloatTensor of size 1]
. Accuracy: 77.0
Iteration: 3000. Loss: Variable containing:
 0.1433
[torch.FloatTensor of size 1]
. Accuracy: 90.5
Iteration: 4000. Loss: Variable containing:
1.00000e-02 *
  9.4418
[torch.FloatTensor of size 1]
. Accuracy: 96.5
Iteration: 5000. Loss: Variable containing:
1.00000e-02 *
  6.2781
[torch.FloatTensor of size 1]
. Accuracy: 97.0
Iteration: 6000. Loss: Variable containing:
1.00000e-02 *
  4.8350
[torch.FloatTensor of size 1]
. Accuracy: 97.0
Iteration: 7000. Loss: Variable containing:
1.00000e-02 *
  3.9872
[torch.FloatTensor of size 1]
. Accuracy: 98.0
Iteration: 8000. Loss: Variable containing:
1.00000e-02 *
  3.4404
[torch.FloatTensor of size 1]
. Accuracy: 98.5
Iteration: 9000. Loss: Variable containing:
1.00000e-02 *
  3.0662
[torch.FloatTensor of size 1]
. Accuracy: 98.5
Iteration: 10000. L

In [24]:
predict_data = pd.read_csv("titanicQ1030.csv",sep=",")
predict_data.head()

,pclass,survived,sex,age,if child,fare,is port,boat,sibsp,parch,is port and man
0,3,NaN,1,0.0,0,3.0,0,0.0,0,0,0
1,3,NaN,0,0.0,0,2.0,0,1.0,2,0,0
2,3,NaN,0,0.0,0,2.0,0,1.0,2,0,0
3,3,NaN,1,0.0,0,2.0,0,1.0,2,0,0
4,3,NaN,0,0.0,0,3.0,0,1.0,0,0,0


In [25]:
#drop=["survived"]
drop=["survived","sibsp","parch","is port"]
pred = predict_data.copy().drop(drop,axis=1).astype(np.float32).as_matrix().reshape(-1,7)

In [26]:
class LR(nn.Module):
    def __init__(self):
        super(LR, self).__init__()
        self.hidden1 = nn.Linear(7, 10)
        self.dropout = nn.Dropout(0.3)
        self.relu1 = nn.ReLU()
        self.hidden2 = nn.Linear(10, 15)
        self.dropout = nn.Dropout(0.3)
        self.relu2 = nn.ReLU()
        self.hidden3 = nn.Linear(15,1)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x2 = F.relu(self.hidden2(x))
        out = self.hidden3(x2)
        return out
model = LR()
model.load_state_dict(torch.load("practice_titanic_model1.pkl"))

In [28]:
x = Variable(torch.from_numpy(pred), requires_grad = True)
ans = model(x)

In [37]:
anslist=[]
for i in range(len(ans)):
    if ans[i].data.numpy()>0.5:
        anslist.append("1")
    else:
        anslist.append("0")

In [38]:
anslist

['0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '1',
 '1',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0'

In [48]:
answer = np.array(anslist)
answer = pd.DataFrame(answer)
answer.to_csv("ans.csv")